In [1]:
import re
import os
import numpy as np
import pandas as pd
from pprint import pprint

import nltk; nltk.download('stopwords')

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models.wrappers import LdaMallet

# spacy for lemmatization
import spacy
import es_core_news_sm

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

# NLTK Stop words
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
## Carga archivo

data = pd.read_csv(r"C:\Users\LENOVO\Downloads\QUEJAS_20200520.csv", sep = "~", engine = "python" )
data.columns

Index(['IDENT_CUENTA', 'ACCOUNT_ID', 'CONTACT_ID', 'DESCRIPCION_DETALLADA',
       'CAUSA', 'SUBCAUSA', 'DPTO_QUEJA'],
      dtype='object')

In [3]:
# NLTK Stop words

stop_words = stopwords.words('spanish')
#stop_words1 = stopwords.words('spanish')
stop_words.extend(['abril','adjunto','adjuntos','apellido','apellidos','apto','argumento','asesor','atlantico','ba','barrio','bello','bogota','bolivar','bucaramanga','caldas','cartagena','cesar','ciudad','contacto','contactar','contrato','celular','correo','cedula','comentarios','documento','direccion','datos','departamento','descripción','detallada','eletronico','electronico','envigado','email','envio','etp','factura','febrero','fecha','fisica','fisico','gmail','hotmail','identificador','japosada','julio','junio','linea','llamada','marzo','mayo','manizales','mes','monteria','motivo','mjgonzal','municipio','nombre','nit','numero','objeto','open','observaciones','oficina','paquete','petición','queja','reclamo','recurso','servicio','sgto','sms','sisback','solicitud','suscriptor','telefono','tigo','titular','tv','uen','usuario','valledupar','ver','virtual'])

In [4]:
# Convert to list
data2 = data.DESCRIPCION_DETALLADA.values.tolist()

In [5]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
data2 = list(sent_to_words(data2))

In [6]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data2, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data2], threshold=100)  

In [7]:
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [8]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [9]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data2)

In [10]:
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

In [11]:
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
## Desde consola de comandos python -m spacy download es_core_news_sm

nlp = es_core_news_sm.load()

In [12]:
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [13]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

In [14]:
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [15]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [16]:
# Print the top ten Keywords in the topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.113*"retirar" + 0.101*"solicitar" + 0.060*"indicar" + 0.052*"pedir" + '
  '0.043*"generar" + 0.040*"servicio" + 0.040*"comunicar" + 0.033*"desear" + '
  '0.031*"favor" + 0.030*"facturar"'),
 (1,
  '0.061*"dar" + 0.028*"presentar" + 0.022*"tecnico" + 0.021*"querer" + '
  '0.020*"respuesta" + 0.020*"deber" + 0.020*"casar" + 0.019*"necesitar" + '
  '0.018*"tiempo" + 0.017*"venir"'),
 (2,
  '0.159*"cuota" + 0.080*"entregar" + 0.049*"pendiente" + 0.047*"financiar" + '
  '0.047*"total" + 0.045*"saldar" + 0.044*"inicial" + 0.042*"valor" + '
  '0.040*"cantidad" + 0.039*"central"'),
 (3,
  '0.167*"acepto" + 0.088*"to" + 0.065*"manifiesto" + 0.061*"respuesta" + '
  '0.057*"enviar" + 0.051*"inconformidad" + 0.050*"clasica_hd" + '
  '0.040*"reclamante" + 0.037*"ilimitado" + 0.033*"mego"'),
 (4,
  '0.082*"fundamentar" + 0.079*"hecho" + 0.071*"tipo" + 0.061*"nombre" + '
  '0.046*"mensaje" + 0.032*"asunto" + 0.021*"archivo" + 0.016*"prestar" + '
  '0.014*"trabajar" + 0.013*"inmediato"'),
 (

In [17]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.


Perplexity:  -7.127286674339411


In [18]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.5655807685039043


In [25]:
# Visualize the topics
pyLDAvis.enable_notebook() 
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis
pyLDAvis.save_html(vis,'lda_Quejas_2_v3.html')

In [30]:
id2word

In [20]:
# Download File: http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
os.environ['MALLET_HOME'] = 'C:\\users\\LENOVO\\Downloads\\mallet-2.0.8'
mallet_path = 'C:/Users/LENOVO/Downloads/mallet-2.0.8/bin/mallet'
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=10, id2word=id2word)

In [21]:
# Show Topics
pprint(ldamallet.show_topics(formatted=False))

[(0,
  [('fundamentar', 0.10686373994160792),
   ('hecho', 0.10485651214128035),
   ('nombre', 0.0826657409385459),
   ('tipo', 0.07525991597237057),
   ('registrar', 0.05451559495834223),
   ('producto', 0.04033148187709179),
   ('abrir', 0.03427864416435235),
   ('asunto', 0.027228868475396996),
   ('archivo', 0.0266369365520188),
   ('tiempo_transcurrido', 0.02573791212703838)]),
 (1,
  [('mesar', 0.08076459445496814),
   ('cambiar', 0.06724903590755149),
   ('descontar', 0.04476311244005406),
   ('ofrecer', 0.03819319421179246),
   ('plan', 0.03192072263882149),
   ('nuevo', 0.029546367758869912),
   ('brindar', 0.028784486852336),
   ('aplicar', 0.02864880943062448),
   ('radicar', 0.025345586048186357),
   ('ofertar', 0.0251994719017278)]),
 (2,
  [('retirar', 0.16432937257409694),
   ('pedir', 0.08054369971595557),
   ('servicio', 0.03614254767534663),
   ('trasladar', 0.034229089521090175),
   ('informar', 0.033659017791195116),
   ('documento', 0.024671713040762606),
   ('acep

In [22]:
# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)


Coherence Score:  0.6886368958377188
